In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sys
sys.path.append("..")

# Start loading the data and getting the vocabulary

In [3]:
df = pd.read_parquet("../data/test.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
vocabulary = set([x for xs in df['elements_1'] for x in xs])

# Check one list contains another list

## Brute force

In [ ]:
from fast_filters.brute_force import brute_force_c2_contains_c1

In [ ]:
%%time
index_brute = brute_force_c2_contains_c1(df, 'elements_1', 'elements_2')

## Binary trick

In [5]:
from fast_filters.binary import Converter, vectorized_not_null_intersection, vectorized_c2_contains_c1

In [8]:
c = Converter(vocabulary={'a', 'b'})

In [14]:
c.convert({})

0

In [6]:
%%time
converter = Converter(vocabulary=vocabulary)
df['elements_1_bin'] = df['elements_1'].map(converter.convert)
df['elements_2_bin'] = df['elements_2'].map(converter.convert)
index_vec = vectorized_not_null_intersection(df, 'elements_1_bin', 'elements_2_bin')

NameError: name 'vocabulary' is not defined

In [9]:
%%time
index_vec = vectorized_c2_contains_c1(df, 'elements_1_bin', 'elements_2_bin')

CPU times: user 3.3 ms, sys: 1.51 ms, total: 4.81 ms
Wall time: 3.78 ms


### how does the integer representation look like?

In [10]:
df.head()

,id,elements_1,elements_2,elements_1_bin,elements_2_bin
0,0,[gzvlwslrnh],[gzvlwslrnh],8,8
1,1,"[wsvkyudbfd, gpxgwigfvd, edjjnoqjyi]","[hdqfajwgbw, wsvkyudbfd, vtqldagyxl]",16402,2192
2,2,[vtqldagyxl],"[wdhltmuqti, edjjnoqjyi]",128,24576
3,3,"[oyizmzfmpr, hdqfajwgbw, wsvkyudbfd, edjjnoqjyi]","[xwyehwnnbw, xczmujmhug, owetlrgmyw, gpxgwigfvd]",18960,5154
4,4,"[xwyehwnnbw, edjjnoqjyi]","[owetlrgmyw, wdhltmuqti]",20480,8224


# Final checks

In [11]:
all(index_brute == index_vec)

True